# Benchmarking RNA-seq DEG Methods with the Dexamethasone Benchmark

In [1]:
# Import libraries
import statistics
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn import preprocessing
import warnings
import requests
import json
import time
import scipy.stats as ss
from itertools import combinations
from rpy2 import robjects
from rpy2.robjects import r, pandas2ri
from maayanlab_bioinformatics.dge.characteristic_direction import characteristic_direction

In [2]:
#pandas2ri.activate()

# Load in Data

Using data from GEO from the study, "The effect of lithium and dexamethasone on fetal rat metatarsal bones transcriptome" 
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE186104 

In [3]:
# Load in data
meta_df = pd.read_csv('GSE186104_series_matrix.txt', sep="\t", index_col=0, dtype=str)
expr_df = pd.read_csv('GSE186104_cross_tabulation_of_gene_expression.txt', index_col=0, sep="\t").sort_index()
meta_class_column_name = 'Sample_characteristics_ch1'
control_name = 'treatment: standard cultivation medium'

meta_df.index = meta_df.index.map(str)
meta_df = meta_df[meta_df.index.isin(expr_df.columns)]

classes = list(meta_df[meta_class_column_name].unique())
classes.remove(control_name)
classes.insert(0, control_name)
meta_df['tmp_class'] = pd.Categorical(meta_df[meta_class_column_name], classes)
meta_df = meta_df.sort_values('tmp_class')
meta_df = meta_df.drop('tmp_class', axis=1)
expr_df = expr_df.loc[:,meta_df.index]
expr_df = expr_df.groupby(expr_df.index).sum()
assert(meta_df.shape[0]==expr_df.shape[1])

dataset = dict()
current_dataset = 'rawdata'
dataset[current_dataset] = expr_df
dataset['dataset_metadata'] = meta_df

In [4]:
low_expression_threshold = 0.3

## Filter out non-expressed genes
expr_df = expr_df.loc[expr_df.sum(axis=1) > 0, :]
## Filter out lowly expressed genes
mask_low_vals = (expr_df > low_expression_threshold).sum(axis=1) > 2
expr_df = expr_df.loc[mask_low_vals, :]
current_dataset += '+filter_genes'
dataset[current_dataset] = expr_df

In [5]:
meta_df

,Sample_title,Sample_characteristics_ch1
Sample_geo_accession,,
GSM5632354,C1: Untreated control replicate 1,treatment: standard cultivation medium
GSM5632355,C2: Untreated control replicate 2,treatment: standard cultivation medium
GSM5632356,C3: Untreated control replicate 3,treatment: standard cultivation medium
GSM5632357,Dex1: Dexamethason treated sample replicate 1,treatment: standard cultivation medium + dexam...
GSM5632358,Dex2: Dexamethason treated sample replicate 2,treatment: standard cultivation medium + dexam...
GSM5632359,Dex3: Dexamethason treated sample replicate 3,treatment: standard cultivation medium + dexam...
GSM5632360,Li1: Lithium treated sample replicate 1,treatment: standard cultivation medium + lithium
GSM5632361,Li2: Lithium treated sample replicate 2,treatment: standard cultivation medium + lithium
GSM5632362,Li3: Lithium treated sample replicate 3,treatment: standard cultivation medium + lithium


In [53]:
expr_df

Sample_geo_accession,GSM5632354,GSM5632355,GSM5632356,GSM5632357,GSM5632358,GSM5632359,GSM5632360,GSM5632361,GSM5632362,GSM5632363,GSM5632364,GSM5632365
gene,,,,,,,,,,,,
A1i3,160,60,196,328,274,287,179,188,343,113,93,119
A2m,32,15,19,62,65,36,28,34,68,23,29,29
A2ml1,1,0,0,0,2,0,3,0,0,2,4,1
A3galt2,80,36,75,68,70,47,63,44,104,55,69,76
A4galt,0,0,2,0,1,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
l7Rn6,392,199,385,174,185,142,189,245,303,269,361,407
mrpl11,374,218,233,144,192,133,156,176,244,210,327,287
mrpl24,611,344,560,222,316,222,286,327,497,369,513,569


In [8]:
def logCPM(data):

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = (data/data.sum())*10**6
        data = data.fillna(0)
        data = np.log2(data+1)

    # Return
    return data

In [9]:
def normalize(dataset, current_dataset, logCPM_normalization, log_normalization, z_normalization, q_normalization):
    normalization = current_dataset
    if logCPM_normalization == True:  
        data = dataset[normalization]
        normalization += '+logCPM'
        dataset[normalization] = logCPM(data)
        
    if log_normalization == True:    
        data = dataset[normalization]
        normalization += '+log'
        dataset[normalization] = log(data)
        
    if z_normalization == True:
        data = dataset[normalization]
        normalization += '+z_norm'    
        dataset[normalization] = data.T.apply(ss.zscore, axis=0).T.dropna()

    if q_normalization == True:
        data = dataset[normalization]
        normalization += '+q_norm'
        dataset[normalization] = qnormalization(data)
    return dataset, normalization

In [10]:
dataset, normalization = normalize(dataset, current_dataset, True, False, True, False)

In [11]:
dataset[normalization]

Sample_geo_accession,GSM5632354,GSM5632355,GSM5632356,GSM5632357,GSM5632358,GSM5632359,GSM5632360,GSM5632361,GSM5632362,GSM5632363,GSM5632364,GSM5632365
gene,,,,,,,,,,,,
A1i3,-0.823915,-1.203701,-0.394838,1.472828,0.950546,1.391266,0.682852,0.464780,0.636785,-0.661800,-1.357623,-1.157182
A2m,-0.850702,-0.913618,-1.407946,1.707599,1.497647,0.792520,0.368323,0.377260,0.752250,-0.662710,-0.757919,-0.902703
A2ml1,-0.294967,-0.772040,-0.772040,-0.772040,0.920040,-0.772040,2.292632,-0.772040,-0.772040,0.670972,1.308982,-0.265415
A3galt2,-0.875900,-1.189250,-0.782604,1.542712,1.049973,0.388160,1.659979,-0.508135,0.933154,-0.559917,-0.820001,-0.838169
A4galt,-0.600760,-0.600760,1.011257,-0.600760,0.792626,2.716666,-0.600760,-0.600760,-0.600760,-0.600760,0.285531,-0.600760
...,...,...,...,...,...,...,...,...,...,...,...,...
l7Rn6,-0.314866,0.016371,0.365216,-0.613465,-1.450318,-1.753019,1.168210,1.839965,-0.805107,0.471590,0.450185,0.625238
mrpl11,0.682765,1.931779,-2.038182,-0.558921,0.423214,-0.539152,0.762820,0.270977,-0.896703,-0.159605,0.889206,-0.768198
mrpl24,0.115012,1.517950,0.140863,-2.218022,-0.175757,-1.392579,1.395951,0.835633,0.106610,-0.338141,-0.004243,0.016723


# Differential Gene Expression
Using code adapted from Bulk RNA-seq Analysis pipeline appyter: https://appyters.maayanlab.cloud/Bulk_RNA_seq/

In [13]:
# Limma not working
"""robjects.r('''limma <- function(rawcount_dataframe, design_dataframe, filter_genes=FALSE, adjust="BH") {
    # Load packages
    suppressMessages(require(limma))
    suppressMessages(require(edgeR))
    # Convert design matrix
    design <- as.matrix(design_dataframe)
    
    # Create DGEList object
    dge <- DGEList(counts=rawcount_dataframe)
    # Filter genes
    if (filter_genes) {
        keep <- filterByExpr(dge, design)
        dge <- dge[keep,]
    }
    # Calculate normalization factors
    dge <- calcNormFactors(dge)
    # Run VOOM
    v <- voom(dge, plot=FALSE)
    # Fit linear model
    fit <- lmFit(v, design)
    # Make contrast matrix
    cont.matrix <- makeContrasts(de=B-A, levels=design)
    # Fit
    fit2 <- contrasts.fit(fit, cont.matrix)
    # Run DE
    fit2 <- eBayes(fit2)
    # Get results
    limma_dataframe <- topTable(fit2, adjust=adjust, number=nrow(rawcount_dataframe))
    
    # Return
    results <- list("limma_dataframe"= limma_dataframe, "rownames"=rownames(limma_dataframe))
    return (results)
}
''')"""

'robjects.r(\'\'\'limma <- function(rawcount_dataframe, design_dataframe, filter_genes=FALSE, adjust="BH") {\n    # Load packages\n    suppressMessages(require(limma))\n    suppressMessages(require(edgeR))\n    # Convert design matrix\n    design <- as.matrix(design_dataframe)\n    \n    # Create DGEList object\n    dge <- DGEList(counts=rawcount_dataframe)\n    # Filter genes\n    if (filter_genes) {\n        keep <- filterByExpr(dge, design)\n        dge <- dge[keep,]\n    }\n    # Calculate normalization factors\n    dge <- calcNormFactors(dge)\n    # Run VOOM\n    v <- voom(dge, plot=FALSE)\n    # Fit linear model\n    fit <- lmFit(v, design)\n    # Make contrast matrix\n    cont.matrix <- makeContrasts(de=B-A, levels=design)\n    # Fit\n    fit2 <- contrasts.fit(fit, cont.matrix)\n    # Run DE\n    fit2 <- eBayes(fit2)\n    # Get results\n    limma_dataframe <- topTable(fit2, adjust=adjust, number=nrow(rawcount_dataframe))\n    \n    # Return\n    results <- list("limma_dataframe"

In [74]:
# Copied from the appyter source code
def get_signatures(control, treatment, dataset, normalization, method, meta_class_column_name, filter_genes):
    tmp_normalization = normalization.replace("+z_norm+q_norm","").replace("+z_norm","")
    raw_expr_df = dataset['rawdata']
    expr_df = dataset[current_dataset]
    if filter_genes == True:
        expr_df = dataset['rawdata+filter_genes']
        
    signatures = dict()

    print(control, treatment)
    cls1_sample_ids = dataset["dataset_metadata"].loc[dataset["dataset_metadata"][meta_class_column_name]==control, :].index.tolist() #control
    cls2_sample_ids = dataset["dataset_metadata"].loc[dataset["dataset_metadata"][meta_class_column_name]==treatment,:].index.tolist() #case
        
    signature_label = " vs. ".join([control, treatment])
       
    # Limma not working
    if method == "limma":
        limma = robjects.r['limma']

        design_dataframe = pd.DataFrame([{'index': x, 'A': int(x in cls1_sample_ids), 'B': int(x in cls2_sample_ids)} for x in raw_expr_df.columns]).set_index('index')

        processed_data = {"expression": raw_expr_df, 'design': design_dataframe}
            
        limma_results = pandas2ri.conversion.rpy2py(limma(pandas2ri.conversion.py2rpy(processed_data['expression']), pandas2ri.conversion.py2rpy(processed_data['design']), filter_genes=filter_genes))
                        
        signature = pd.DataFrame(limma_results[0])
        signature.index = limma_results[1]
        signature = signature.sort_values("t", ascending=False)
            
    elif method == "logFC":
        values = []
        for i in range(len(expr_df)):
            case_mean = statistics.mean(expr_df.iloc[i][cls2_sample_ids])
            control_mean = statistics.mean(expr_df.iloc[i][cls1_sample_ids])
            if case_mean == 0 or control_mean == 0:
                values.append('NA')
            else:
                values.append(math.log(case_mean/control_mean,2))

        signature = pd.DataFrame(values, columns = ['logFC'])
        signature.index = expr_df.index
        signature = signature[signature['logFC'] != 'NA']
        signature = df.sort_values("logFC", ascending=False)
            
    elif method == "characteristic_direction":
        signature = characteristic_direction(dataset[tmp_normalization].loc[:, cls1_sample_ids], dataset[normalization].loc[:, cls2_sample_ids], calculate_sig=True)
        signature = signature.sort_values("CD-coefficient", ascending=False)
            
    signatures[signature_label] = signature

    return signatures

In [15]:
# Set method in this variable
diff_gex_method = 'characteristic_direction'
control = 'treatment: standard cultivation medium'
treatment = 'treatment: standard cultivation medium + dexamethason'

signatures = get_signatures(control, treatment, dataset, normalization, diff_gex_method, meta_class_column_name, True)

treatment: standard cultivation medium treatment: standard cultivation medium + dexamethason


In [49]:
cd_df = signatures['treatment: standard cultivation medium vs. treatment: standard cultivation medium + dexamethason']
cd_df

,CD-coefficient,Significance
gene,,
Rbp7,0.016617,-0.088503
LOC102556144,0.016617,-0.139658
Selp,0.016136,-0.073505
LOC102550122,0.016067,-0.135190
Slc18a3,0.016039,-0.066825
...,...,...
Col1a1,-0.024673,-0.067146
Col11a2,-0.025156,-0.066973
Col9a2,-0.025866,-0.067782


In [75]:
diff_gex_method = 'logFC'
signatures = get_signatures(control, treatment, dataset, normalization, diff_gex_method, meta_class_column_name, True)

treatment: standard cultivation medium treatment: standard cultivation medium + dexamethason


In [76]:
fc_df = signatures['treatment: standard cultivation medium vs. treatment: standard cultivation medium + dexamethason']
fc_df

,logFC
gene,
Cd163,8.885696
Orm1,6.087463
Calb2,5.260528
Cd101,4.813069
LOC100909954,4.778442
...,...
Pck1,-5.247928
Asb2,-5.426265
LOC102550863,-5.554589


In [77]:
cd_up_200_genes = list(cd_df.head(200).index)
cd_down_200_genes = list(cd_df.tail(200).index)
cd_both_400_genes = cd_up_200_genes + cd_down_200_genes
logFC_up_200_genes = list(fc_df.head(200).index)
logFC_down_200_genes = list(fc_df.tail(200).index)
logFC_both_400_genes = logFC_up_200_genes + logFC_down_200_genes

gene_lists = [cd_up_200_genes, cd_down_200_genes, cd_both_400_genes, logFC_up_200_genes, logFC_down_200_genes, logFC_both_400_genes]
list_names = ["CD Method: 200 Upregulated Genes", "CD Method: 200 Downregulated Genes", "CD Method: Combined 400 Genes",
             "LogFC Method: 200 Upregulated Genes", "LogFC Method: 200 Downregulated Genes", "LogFC Method: Combined 400 Genes"]

# Enrichment Analysis with Enrichr

In [79]:
# Function to get Enrichr Results
def Enrichr_API(enrichr_gene_list, all_libraries):


    all_ranks = []
    all_terms = []
    all_pvalues =[] 
    all_adjusted_pvalues = []
    library_success = []
    short_id = ''

    for library_name in all_libraries : 
        ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/addList'
        genes_str = '\n'.join(enrichr_gene_list)
        description = 'Example gene list'
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }

        response = requests.post(ENRICHR_URL, files=payload)
        if not response.ok:
            raise Exception('Error analyzing gene list')

        data = json.loads(response.text)
        time.sleep(0.5)
        ENRICHR_URL = 'http://amp.pharm.mssm.edu/Enrichr/enrich'
        query_string = '?userListId=%s&backgroundType=%s'
        user_list_id = data['userListId']
        short_id = data["shortId"]
        gene_set_library = library_name
        response = requests.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
         )
        if not response.ok:
            raise Exception('Error fetching enrichment results')
        try:
            data = json.loads(response.text)
            results_df  = pd.DataFrame(data[library_name])
            all_ranks.append(list(results_df[0]))
            all_terms.append(list(results_df[1]))
            all_pvalues.append(list(results_df[2]))
            all_adjusted_pvalues.append(list(results_df[6]))
            library_success.append(library_name)
        except:
            print('Error for ' + library_name + ' library')

    return([all_ranks,all_terms,all_pvalues,all_adjusted_pvalues,str(short_id),library_success])

In [80]:
results = []
for gene_list in gene_lists:
    result = Enrichr_API(gene_list, ['ChEA_2016'])
    results.append(result)

In [81]:
# Extract NR3C1 rankings
# Initialize lists for storing NR3C1 information
gene_set = []
names = []
ranks = []
p_val = []

# Iterate over each result
for i in range(len(results)):
    # Within each gene set, iterate over the transcription factor names
    for j in range(len(results[i][1][0])):
        # If NR3C1 is found, add the information to the lists
        if 'NR3C1' in results[i][1][0][j]:
            names.append(results[i][1][0][j])
            ranks.append(results[i][0][0][j])
            p_val.append(results[i][2][0][j])
            gene_set.append(list_names[i])

print(gene_set, names, ranks, p_val)

['CD Method: 200 Upregulated Genes', 'CD Method: 200 Upregulated Genes', 'CD Method: 200 Downregulated Genes', 'CD Method: 200 Downregulated Genes', 'CD Method: Combined 400 Genes', 'CD Method: Combined 400 Genes', 'LogFC Method: 200 Upregulated Genes', 'LogFC Method: 200 Upregulated Genes', 'LogFC Method: 200 Downregulated Genes', 'LogFC Method: 200 Downregulated Genes', 'LogFC Method: Combined 400 Genes', 'LogFC Method: Combined 400 Genes'] ['NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human', 'NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human', 'NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human', 'NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human', 'NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human', 'NR3C1 23031785 ChIP-Seq PC12 Mouse', 'NR3C1 21868756 ChIP-Seq MCF10A Human'] [90, 117, 348, 416, 202, 283, 42, 83, 126, 223, 65, 137] [0.955186110169955, 0.97292

In [82]:
# Create a display dataframe
df = pd.DataFrame(list(zip(gene_set, names, ranks, p_val)),
                 columns = ['Gene Set', 'Name','Rank','p-value'])
df

,Gene Set,Name,Rank,p-value
0,CD Method: 200 Upregulated Genes,NR3C1 23031785 ChIP-Seq PC12 Mouse,90,0.955186
1,CD Method: 200 Upregulated Genes,NR3C1 21868756 ChIP-Seq MCF10A Human,117,0.972929
2,CD Method: 200 Downregulated Genes,NR3C1 23031785 ChIP-Seq PC12 Mouse,348,0.436921
3,CD Method: 200 Downregulated Genes,NR3C1 21868756 ChIP-Seq MCF10A Human,416,0.582927
4,CD Method: Combined 400 Genes,NR3C1 23031785 ChIP-Seq PC12 Mouse,202,0.823345
5,CD Method: Combined 400 Genes,NR3C1 21868756 ChIP-Seq MCF10A Human,283,0.915026
6,LogFC Method: 200 Upregulated Genes,NR3C1 23031785 ChIP-Seq PC12 Mouse,42,0.572154
7,LogFC Method: 200 Upregulated Genes,NR3C1 21868756 ChIP-Seq MCF10A Human,83,0.804352
8,LogFC Method: 200 Downregulated Genes,NR3C1 23031785 ChIP-Seq PC12 Mouse,126,0.901381
9,LogFC Method: 200 Downregulated Genes,NR3C1 21868756 ChIP-Seq MCF10A Human,223,0.972929


# Comparing methods

In [84]:
# Compare rankings/methods